### Installations

In [1]:
#choose a minimal T4 GPU execution
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


restart your session

In [ ]:
#check GPU config
!nvidia-smi

Wed Jul 17 12:29:05 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   45C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from google.colab import userdata
# Defined in the secrets tab in Google Colab (key icon)
hf_token = userdata.get('HF_TOKEN')

### Function to test model

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, TextStreamer

torch.random.manual_seed(0)
model = AutoModelForCausalLM.from_pretrained(
    "jpacifico/Chocolatine-3B-Instruct-DPO-Revised",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("jpacifico/Chocolatine-3B-Instruct-DPO-Revised")

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [19]:
def generate_response(prompt, max_new_tokens=500, temperature=0.0):
    messages = [
        {"role": "system", "content": "Tu es un assistant IA baptisé Chocolatine. Ta mission est de fournir des informations sûres, éthiques et précises à l'utilisateur."},
        {"role": "user", "content": prompt},
    ]
    pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    )

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "return_full_text": False,
        "temperature": temperature,
        "do_sample": False,
    }

    output = pipe(messages, **generation_args)
    return output[0]['generated_text']

In [21]:
user_prompt = "Donne moi la recette de la ratatouille."
print(generate_response(user_prompt))

 Voici une recette traditionnelle de ratatouille, un plat provençal à base de légumes.


Ingrédients:

- 1 aubergine

- 1 courgette

- 2 poivrons (un rouge et un jaune)

- 2 tomates mûres

- 1 oignon

- 2 gousses d'ail

- Huile d'olive

- Herbes de Provence

- Sel et poivre


Préparation:

1. Lavez tous les légumes. Coupez l'aubergine et la courgette en dés de 2 cm.

2. Émincez l'oignon et l'ail.

3. Dans une grande poêle, faites chauffer de l'huile d'olive et faites revenir l'oignon et l'ail jusqu'à ce qu'ils soient translucides.

4. Ajoutez les poivrons et faites-les cuire jusqu'à ce qu'ils commencent à ramollir.

5. Incorporez les tomates coupées en dés et laissez mijoter.

6. Ajoutez les aubergines et courgettes, puis versez les herbes de Provence, le sel et le poivre.

7. Laissez mijoter à feu doux pendant environ 30 minutes, en remuant de temps en temps.

8. Servez chaud, accompagné de riz ou de pain.


In [22]:
def stream_response(prompt, max_new_tokens=500, temperature=0.0):
    messages = [
        {"role": "system", "content": "Tu es un assistant IA baptisé French-Orca. Ta mission est de fournir des informations sûres, éthiques et précises à l'utilisateur."},
        {"role": "user", "content": prompt},
    ]
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    generation_args = {
        "max_new_tokens": max_new_tokens,
        "temperature": temperature,
        "do_sample": False,
        "streamer": streamer,
    }

    pipe(messages, **generation_args)

In [23]:
user_prompt = "Donne moi la recette de la ratatouille."
stream_response(user_prompt)

Pour préparer une ratatouille traditionnelle, suivez ces étapes :


Ingrédients :

- 1 aubergine

- 1 courgette

- 2 poivrons (un rouge et un jaune)

- 2 tomates mûres

- 1 oignon

- 2 gousses d'ail

- Huile d'olive

- Herbes de Provence

- Sel et poivre


1. Lavez tous les légumes. Coupez l'aubergine et la courgette en dés de 2 cm.

2. Émincez l'oignon et l'ail. Coupez les poivrons en lanières.

3. Dans une grande poêle, faites chauffer de l'huile d'olive à feu moyen. Ajoutez l'oignon et l'ail, faites-les revenir jusqu'à ce qu'ils soient tendres.

4. Ajoutez les poivrons et faites-les cuire pendant 5 minutes.

5. Incorporez les tomates coupées en dés et laissez mijoter.

6. Ajoutez les aubergines et la courgette, assaisonnez avec du sel, du poivre et des herbes de Provence.

7. Laissez mijoter à feu doux pendant environ 30 minutes, en remuant de temps en temps.

8. Servez chaud, accompagné de riz ou de pain.
